In [159]:
import pandas as pd

In [160]:
_2021 = pd.read_csv('_2021.csv', encoding='utf-8')
_2021['Injeção na rede (kWh)'].fillna("None", inplace=True)

_2022 = pd.read_csv('_2022.csv', encoding='utf-8')
_2022['Injeção na rede (kWh)'].fillna("None", inplace=True)

treino = pd.concat([_2021, _2022], ignore_index=True)
treino.drop('Unnamed: 0', axis=1, inplace=True)

treino.drop('temp', axis=1, inplace=True)
treino.drop('feels_like', axis=1, inplace=True)
treino.drop('temp_min', axis=1, inplace=True)
treino.drop('temp_max', axis=1, inplace=True)
treino.drop('pressure', axis=1, inplace=True)
treino.drop('humidity', axis=1, inplace=True)
treino.drop('wind_speed', axis=1, inplace=True)
treino.drop('clouds_all', axis=1, inplace=True)
treino.drop('weather_description', axis=1, inplace=True)

In [161]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11016 entries, 0 to 11015
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Normal (kWh)             11016 non-null  float64
 1   Horário Económico (kWh)  11016 non-null  float64
 2   Autoconsumo (kWh)        11016 non-null  float64
 3   Injeção na rede (kWh)    11016 non-null  object 
 4   Data & Hora              11016 non-null  object 
dtypes: float64(3), object(2)
memory usage: 430.4+ KB


In [162]:
treino.head()

,Normal (kWh),Horário Económico (kWh),Autoconsumo (kWh),Injeção na rede (kWh),Data & Hora
0,0.0,0.0,0.0,None,2021-09-29 00:00:00
1,0.0,0.0,0.0,None,2021-09-29 01:00:00
2,0.0,0.0,0.0,None,2021-09-29 02:00:00
3,0.0,0.0,0.0,None,2021-09-29 03:00:00
4,0.0,0.0,0.0,None,2021-09-29 04:00:00


In [163]:
teste = pd.read_csv('energia_2023.csv', encoding='utf-8')
teste.drop('Unnamed: 0', axis=1, inplace=True)

In [164]:
import numpy as np

import ants

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score

from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import seaborn as sns

In [165]:
treino.drop('Data & Hora', axis=1, inplace=True)

In [166]:
teste.drop('Data & Hora', axis=1, inplace=True)

In [167]:
'''#Hot
weather_description = pd.get_dummies (treino['weather_description'], drop_first=True)
treino.drop('weather_description', axis=1, inplace=True)
treino = pd.concat ([treino, weather_description], axis=1)

#label'''

"#Hot\nweather_description = pd.get_dummies (treino['weather_description'], drop_first=True)\ntreino.drop('weather_description', axis=1, inplace=True)\ntreino = pd.concat ([treino, weather_description], axis=1)\n\n#label"

In [168]:
'''#Hot
weather_description = pd.get_dummies (teste['weather_description'], drop_first=True)
teste.drop('weather_description', axis=1, inplace=True)
teste = pd.concat ([teste, weather_description], axis=1)

#label'''

"#Hot\nweather_description = pd.get_dummies (teste['weather_description'], drop_first=True)\nteste.drop('weather_description', axis=1, inplace=True)\nteste = pd.concat ([teste, weather_description], axis=1)\n\n#label"

In [169]:
# Injeção na rede (kWh) troca do tipo dos valores
treino['Injeção na rede (kWh)'].replace({'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}, inplace=True)
print(treino['Injeção na rede (kWh)'].value_counts())

Injeção na rede (kWh)
0    7777
3    1103
2    1098
4     606
1     432
Name: count, dtype: int64


In [170]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11016 entries, 0 to 11015
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Normal (kWh)             11016 non-null  float64
 1   Horário Económico (kWh)  11016 non-null  float64
 2   Autoconsumo (kWh)        11016 non-null  float64
 3   Injeção na rede (kWh)    11016 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 344.4 KB


In [171]:
X = treino.drop(['Injeção na rede (kWh)'], axis=1)
y = treino['Injeção na rede (kWh)'].to_frame()

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.30, random_state=2021)

In [172]:
parameters = { 'min_child_weight': [5, 6, 7]
             , 'colsample_bytree': [0, 0.2]
             , 'max_depth': [3, 4, 5]
             }

_xgb = xgb.XGBClassifier(objective='reg:squarederror')
clf = GridSearchCV (estimator= _xgb, param_grid= parameters, cv= 5, refit= True, verbose= 3, error_score='raise')

In [173]:
best_clf = clf.fit(X_train, y_train.values.ravel())
_xgb.fit (X_train, y_train.values.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END colsample_bytree=0, max_depth=3, min_child_weight=5;, score=0.833 total time=   0.1s
[CV 2/5] END colsample_bytree=0, max_depth=3, min_child_weight=5;, score=0.846 total time=   0.2s
[CV 3/5] END colsample_bytree=0, max_depth=3, min_child_weight=5;, score=0.851 total time=   0.1s
[CV 4/5] END colsample_bytree=0, max_depth=3, min_child_weight=5;, score=0.834 total time=   0.1s
[CV 5/5] END colsample_bytree=0, max_depth=3, min_child_weight=5;, score=0.835 total time=   0.0s
[CV 1/5] END colsample_bytree=0, max_depth=3, min_child_weight=6;, score=0.835 total time=   0.2s
[CV 2/5] END colsample_bytree=0, max_depth=3, min_child_weight=6;, score=0.846 total time=   0.1s
[CV 3/5] END colsample_bytree=0, max_depth=3, min_child_weight=6;, score=0.848 total time=   0.1s
[CV 4/5] END colsample_bytree=0, max_depth=3, min_child_weight=6;, score=0.833 total time=   0.1s
[CV 5/5] END colsample_bytree=0, max_depth=3, min_child_w

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [174]:
def replace_int_with_string(value):
    mapping = {0: "None"
          , 1: "Low"
          , 2: "Medium"
          , 3: "High"
          , 4: "Very High"
    }
    return mapping[value]

In [175]:
xgb_predictions = _xgb.predict(teste)
xgb_predictions = map(lambda x: replace_int_with_string(x), xgb_predictions)

In [176]:
best_predictions = best_clf.predict(teste)
best_predictions = map(lambda x: replace_int_with_string(x), best_predictions)

In [177]:
import csv

with open('xgboost-energia_2021_[5, 6, 7]_[0, 0.2]_raise.csv', 'w', newline='') as file:
    writer = csv.writer( file )
    writer.writerow(["RowId", "Result"])
    
        
    _id = 1
    for prediction in xgb_predictions:
        writer.writerow([_id, prediction])
        _id +=1

with open('xgboost-energia-best_2021_[5, 6, 7]_[0, 0.2]_raise.csv', 'w', newline='') as file:
    writer = csv.writer( file )
    writer.writerow(["RowId", "Result"])
    
        
    _id = 1
    for prediction in best_predictions:
        writer.writerow([_id, prediction])
        _id +=1